In [7]:
"""
Panel-1 
Define config variable; Select project; Select Image;
Gets datasets ids
Author: AAV
vamps2.mbl.edu
"""
import os,sys
import requests
from bs4 import BeautifulSoup   # parser for html
conn = {'user':'guest',
        'passwd':'guest',
         # vamps:             https://vamps2.mbl.edu
         # vampsdev (private) http://vampsdev.jbpc-np.mbl.edu:8124 
         # localhost:         http://localhost:3000 
        'hosturl':'http://localhost:3000'
       }

s = requests.Session()
r = s.post(conn['hosturl']+'/users/login', data={'username':conn['user'], 'password':conn['passwd']})
#r = s.get('http://localhost:3000/users/logout')  <-- needed?
config = {
    "api":"1",
    "source":"VAMPS-API",
    "update_data":1,
    "normalization":"none",              # none, maximum, frequency
    "selected_distance":"jaccard",  # morisita-horn, jaccard, kulczynski, canberra bray-curtis
    "tax_depth":"klass",                 # domain, phylum, klass, order, family, genus, species, strain
    "domains":["Archaea","Bacteria","Eukarya","Organelle","Unknown"],   #["Archaea","Bacteria","Eukarya","Organelle","Unknown"],
    "include_nas":"yes",                  # yes or no
    "min_range":0,                        # integer 0-99
    "max_range":100,                      # integer 1-100

      # Must be a valid project - with correct permissions for the above user.
    'project':'ICM_SPO_Ev9',   # localhost tests: KCK_LSM_TBS(private) or ICM_SPO_Ev9 or ICM_LCY_Bv6   
                                # vamps2: AB_SAND_Bv6 AFP_MVCO_Bv6(private) DCO_BRG_Bv6v4

      # Currently avalable: "dheatmap", "piecharts", "barcharts", "counts_matrix", "metadata_csv", "adiversity"
      # FUTURE:            "metadata_table", "fheatmap", "dendrogram01", "dendrogram03", 
      # FUTURE:            "pcoa", "pcoa3d", "geospatial" 
    'image':'dendrogram',
    # possible values   "fasta", "metadata_csv", "metadata_table"   <-- not implemented
    #'file' :'metadata-csv'
}
# Get project ids:
r = s.post(conn['hosturl']+'/api/get_dids_from_project', timeout=15, data=config)  
config['ds_order'] = r.text
print(config['ds_order'])

[29,30,31,32,33,34,35,36,37,38,39,40,41,42,43,44]


In [8]:
"""
Panel-2
Creates remote configuration and gets ts (file_prefix) 
"""
# Get timestamp (filename prefix):
r = s.post(conn['hosturl']+'/visuals/view_selection', timeout=15, data=config)
soup = BeautifulSoup(r.text, "lxml")  # html5lib  lxml html.parser

# I placed this on view_selection page:  <div id='ts_for_bs' style='visibility:hidden'><%= pi.ts %></div>
ts = soup.find(id="ts_for_bs").string
print(ts)

guest_1499888927787


In [9]:
"""
Panel-3
Saves matrx_file which is integral to vamps images:
Saves image file
"""
import json
biom_matrix_file = ts+'_count_matrix.biom'
url = conn['hosturl']+"/"+biom_matrix_file
response = requests.get(url, stream=True)
response.raise_for_status()
out_file = biom_matrix_file
with open(out_file, "wb") as handle:
    for block in response.iter_content(1024):
        handle.write(block)


r = s.post(conn['hosturl']+'/api/create_image', timeout=30, data=config)

try:
    result = json.loads(r.text)
except:
    print(r.text)
    sys.exit()
local_filename = result['filename']
return_result = result['html']
print(local_filename)
remote_file_name = conn['hosturl']+"/"+local_filename

r = requests.get(remote_file_name, stream=True)
with open(local_filename, 'wb') as f:
    f.write(r.content)
print 'Done writing local file:',local_filename

guest_1499888927787_newick.tre
Done writing local file: guest_1499888927787_newick.tre


In [29]:
"""
Show image
"""
ready_images = ["dheatmap", "piecharts", "barcharts", "counts_matrix", "metadata_csv", "adiversity","dendrogram"]
from IPython.core.display import display, HTML
if config['image'] in ready_images:
    if config['image'] == 'metadata_csv':
        print(return_result)
    elif config['image'] == 'dendrogram':
        from ete3 import Tree, TreeStyle
        circular_style = TreeStyle()
        circular_style.mode = "c" # draw tree in circular mode
        circular_style.scale = 300
        rooted_tree = Tree( return_result )
        #print rooted_tree
        file = ts+'_tree.svg'
        rooted_tree.render(file, w=80, units="mm", tree_style=circular_style)
        display(HTML(open(file).read()))
    else:
        
        display(HTML("<style>.container { width:100% !important; }</style>"+return_result))
else:
    print('not implemented yet')